In [1]:
import numpy as np
import time
import os

import matplotlib.pyplot as plt
import pandas as pd
import random

import json

import keras
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import models

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = 'false'
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], False)

DEBUG = False

2022-02-01 18:15:34.416360: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-01 18:15:34.442784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-01 18:15:34.443036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
column_names = ['timestamp','open','close','high','low','volume','adosc','atr','macd','macd_signal','macd_hist','mfi','upper_band','middle_band','lower_band','rsi','difference_low_high','difference_open_close','target']
files = '/home/joren/Coding/cryptodata/Normalized_labelled/'
modelfiles = ['model_small_15']
frame_sizes = [360]

In [3]:
def filenames(dir):
    filenames = []
    for file in os.listdir(dir):
        filename = os.fsdecode(file)
        filenames.append(filename)
    return filenames

field_info = [
    { "type": np.uint64, "count": 1 },
    { "type": np.double, "count": 17 },
    { "type": np.int64, "count": 1 }
]
BYTES_EIGHT = 8

def read_bin_full_file(file):
    f = open(file, 'rb')
    b = f.read(-1)

    BYTES_TO_READ = 0
    for field in field_info:
        BYTES_TO_READ += BYTES_EIGHT * field["count"]

    data = []
    BYTES_READ = 0
    for i in range(0, int(os.path.getsize(file) / BYTES_TO_READ)):
        row = []

        for idx, field in enumerate(field_info):
            row += np.frombuffer(b, dtype=field["type"], count=field["count"], offset=BYTES_READ).tolist()

            BYTES_READ += BYTES_EIGHT * field["count"]

        data.append(row)
    return np.array(data, dtype=np.float16)

# train-test split
def df_split(df, frame_size):  
    X = df.drop(columns=['timestamp','target'], axis=0).to_numpy()
    Y = df['target'].to_numpy()

    X_test = []
    y_test = []
    for i in range(frame_size, X.shape[0]): 
        X_test.append(X[i-frame_size:i])
        y_test.append(Y[i])
    X_test, y_test = np.array(X_test, dtype=np.float16), np.array(y_test, dtype=np.float16)

    return X_test, y_test

In [4]:
for modelfile_index in range(len(modelfiles)):
    modelfile = './models/'+modelfiles[modelfile_index]
    frame_size = frame_sizes[modelfile_index]

    # load model
    model = models.load_model(modelfile)

    i = 0
    total_percentage_raw = 0
    negatives = []
    length = 0

    for file in filenames(files):
        ## prepare data for prediction
        data = read_bin_full_file(files+file)
        df2 = pd.DataFrame(data, columns=column_names)
        # df2.fillna(0, inplace=True)
        startloc = round(len(df2)*0.98)
        df = df2.iloc[startloc:]
        if len(df) < frame_size*5:
            continue
        i += 1


        X_test, y_test = df_split(df, frame_size)

        ## predict
        y_pred = np.array(model.predict(X_test), dtype=np.float16)

        y_pred = np.argmax(y_pred, axis=-1)
        y_pred = y_pred.flatten()

        df = df.iloc[frame_size:]
        length += len(df)/60
        df['predictions'] = np.array(y_pred)

        # calculate profit/loss
        temp_percentage = 0
        percentage = 0
        position = 0

        for index, row in df.iterrows():

            if position == 1:
                percentage += row['close']

            if row['predictions'] == 1:
                position = 1
            elif row['predictions'] == 2:
                position = 0
                total_percentage_raw += percentage
                temp_percentage += percentage
                percentage = 0

        if temp_percentage < 0:
            negatives.append({
                'file': file,
                'result': temp_percentage
            })
            print(f"negatives: {negatives}")

        dictionary ={
            "model" : modelfile,
            "files_processed": i,
            "average_percentage" : total_percentage_raw/i,
            "negative_results": negatives,
            "average_length_in_hours": length/i
        }

        # new_object = json.dumps(dictionary, indent = 4)
        # json_object['results'].append(dictionary)
        json_object = json.dumps(dictionary, indent = 4)
        with open(f"test_results/{modelfiles[modelfile_index]}_result.json", "a") as outfile:
            outfile.write(json_object)
        

2022-02-01 18:15:34.818212: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-01 18:15:34.818921: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-01 18:15:34.819121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-01 18:15:34.819260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

KeyboardInterrupt: 